In [ ]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf

Read the mesh

In [ ]:
V, F = igl.read_triangle_mesh("mesh.obj")
mp.plot(V, F)

Get mesh bounding box and center

In [ ]:
minn = np.min(V, axis=0)
maxx = np.max(V, axis=0)

center = (minn+maxx)/2

Function to select sidesets:

- Sideset 2: $z$-component close to the bottom boundary `minn[2]` and larger than $y$ of center
- Sideset 3: $z$-component close to the bottom boundary `minn[2]` and smaller than $y$ of center

In [1]:
def sideset(p):
    if abs(p[2] - minn[2]) < 0.5:
        if p[1] > center[1]:
            return 2
        else:
            return 3
    return 1

Wildmeshing tetrahedralization

In [ ]:
wm.tetrahedralize("mesh.obj", "out.mesh", mute_log=True)

Loading the mesh

In [ ]:
solver = pf.Solver()
solver.load_mesh_from_path("out.mesh")

Using the previous function as sidesets and visualization for confirmation

In [ ]:
solver.set_boundary_side_set_from_bary(sideset)

ps, ts, s = solver.get_boundary_sidesets()

col = np.zeros_like(s)
col[s==2] = 2
col[s==3] = 3

mp.plot(ps, ts, col, shading={"wireframe": False})

Setup the problem

In [ ]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)


problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [0, 0.5, 0])

settings.set_problem(problem)

Solve!

In [ ]:
solver.settings(settings)
solver.solve()

Visualization of displacement and stresses

In [ ]:
p, t, d = solver.get_sampled_solution()
m, _ = solver.get_sampled_mises_avg()

In [ ]:
mp.plot(p+d, t, m, shading={"wireframe": False})